# Caso: Análisis de los Logs de Navegación de la web de un Banco

Utilizamos una muestra del Log de Navegación del sitio web de un Banco
##### Clientes analizados: 6,576
##### Horizonte de tiempo: Del 12-Mar-2018 al 21-May-2018 (Poco menos de 3 meses)

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx
import time

In [ ]:
df = pd.read_csv('../data/weblog.csv')
df.head()

In [ ]:
df.info()

Revisamos la información del DataFrame

In [ ]:
df=df.sort_values(['CUSTOMER_ID', 'TD_TIMECODE']).reset_index(drop=True)
df.head()

In [ ]:
seq = pd.DataFrame()
seq = pd.DataFrame(columns = ['customer', 'inicio', 'fin'])
for i in range(len(df)-1):
    if df.iloc[i,1] == df.iloc[i+1,1]:
        try:
            user = df.iloc[i,1]
            start = df.iloc[i,3]
            end = df.iloc[i+1,3]
            seq = seq.append({'customer' : user, 'inicio' : start, 'fin' : end}, ignore_index = True)
        except Exception:
            pass

In [ ]:
seq.head()

Agrupamos por las páginas visitadas y contamos cuántos clientes hicieron ese salto

In [ ]:
h=seq.groupby(['inicio', 'fin'], as_index=False).count()
h.head()

#### Creamos el Grafo de Navegación Web

Transformamos el DataFrame en un Grafo dirigido con pesos (cantidad de llamadas y duración)

In [ ]:
G = nx.from_pandas_edgelist(h, source='inicio', target='fin', edge_attr=['customer'], create_using=nx.DiGraph())
print(nx.info(G))

#### Grado

Guardar el grado de cada nodo en un diccionario

In [ ]:
gdeg=G.degree(weight='customer')
dict(gdeg)

Visualizamos la red considerando el grado y el peso de la relación

In [ ]:
plt.figure(figsize=(16,9)) 
pos=nx.circular_layout(G)
edges = G.edges()
weights = [G[u][v]['customer']/100 for u,v in edges]
nx.draw_networkx(G, width=weights, pos=pos, node_size=[val for(node,val)in gdeg])
plt.show()

In [ ]:
##pip install netgraph

In [ ]:
from netgraph import InteractiveGraph
plt.figure(figsize=(16,9)) 
plot_instance = InteractiveGraph(G, node_size=4, node_labels=True, node_label_offset=0.1, node_label_fontdict=dict(size=20),
    edge_width=2, arrows=True)
plt.show()

In [ ]:
indeg=G.in_degree(weight='customer')
dict(indeg)

In [ ]:
outdeg=G.out_degree(weight='customer')
dict(outdeg)

IMPORTANTE: Tenemos dos puntos de cierre: 'Purchase' que es COMPRA y 'Product Return' que es DEVOLUCIÓN 

#### Betweenness

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G,weight='customer')
dict(betweenness_centrality)

In [ ]:
plt.figure(figsize=(16,9)) 
sizes = dict(betweenness_centrality).values()
nx.draw_networkx(G, width=weights, pos=pos, node_size=[v*10000 for v in sizes])
plt.show()

#### PageRank

In [ ]:
pagerank_centrality =nx.pagerank(G,weight='customer')
dict(pagerank_centrality)

In [ ]:
plt.figure(figsize=(16,9)) 
sizes = dict(pagerank_centrality).values()
nx.draw_networkx(G, width=weights, pos=pos, node_size=[v*20000 for v in sizes])
plt.show()

Elaborado por Luis Cajachahua bajo licencia MIT (2021)